In [91]:
import pandas as pd
import json
from collections import Counter

# Questions:
1. What percentage of bills that became law passed their legislative votes with most dems and a few GOP reps?
2. What percent pass with support from both caucuses?

In [3]:
def read_json(path):
    with open(path) as file:
        return json.load(file)

In [4]:
%ls './../app/src/data'

leg-roster-2019.json          scrape-2019-02-23.json
mtleg-2017.json               scrape-2019-03-01-all.json
mtleg-2019.json               scrape-2019-all.json
scrape-2017-bills.json        scrape-2019-bills.json
scrape-2017-votes.json        scrape-2019-votes-TEST.json
scrape-2019-02-21.json        scrape-2019-votes.json
scrape-2019-02-23-bills.json  test.json
scrape-2019-02-23-votes.json


In [67]:
raw = read_json('./../app/src/data/mtleg-2017.json')
bills = raw['bills']
votes = raw['votes']
lawmakers = raw['lawmakers']

In [8]:
for key in raw:
    print(key)

updateDate
votes
bills
lawmakers


In [15]:
# statuses = [bill['extras']['status'] for bill in bills]

In [161]:
passed = list(filter(lambda bill: bill['extras']['status'] == 'Became Law', bills))

KeyError: 'bill'

In [68]:
len(passed)

446

In [34]:
lawmakers[0]

{'last_name': 'ABBOTT',
 'first_name': 'KIM',
 'party': 'D',
 'district': 'HD 83',
 'district_num': 83,
 'Addr_Ln_1': 'PO BOX 1752',
 'City': 'HELENA',
 'StateProvince': 'MT',
 'PostalCode': 59624,
 'chamber': 'house',
 'name': 'ABBOTT, KIM',
 'laws_vote_name': 'Abbott, Kim',
 'percentVotesWithMajority': 0.7813455657492355,
 'percentVotesWithGopCaucus': 0.654434250764526,
 'percentVotesWithDemCaucus': 0.9571865443425076}

In [30]:
 len(votes)

4561

In [105]:
bills[0]

{'legislative_session': '2017',
 'identifier': 'SB 23',
 'title': 'Allow in-house counsel to be deputy commissioner of political practices',
 'from_organization': '~{"classification": "upper"}',
 'classification': ['bill'],
 'subject': ['Elections  (see also: Ballot Issues)'],
 'abstracts': [],
 'other_titles': [],
 'other_identifiers': [],
 'actions': [{'description': 'Draft Request Received',
   'date': '2016-09-02',
   'organization_id': '~{"classification": "legislature"}',
   'classification': [],
   'related_entities': [],
   'extras': {'committee': ''}},
  {'description': 'Draft to Requester for Review',
   'date': '2016-10-25',
   'organization_id': '~{"classification": "legislature"}',
   'classification': [],
   'related_entities': [],
   'extras': {'committee': ''}},
  {'description': 'Draft to Requester for Review',
   'date': '2016-10-25',
   'organization_id': '~{"classification": "legislature"}',
   'classification': [],
   'related_entities': [],
   'extras': {'committe

In [152]:
def get_bill_votes(bill):
    bill_id = bill['_id']
    return list(filter(lambda vote: vote['bill'] == bill_id, votes)) 

def categorize_votes(vote):
    if (vote['gopCaucus']['caucus'] == 'yes' and vote['demCaucus']['caucus'] == 'yes'):
        vote_class = 'both support'
    elif (vote['gopCaucus']['caucus'] == 'no' and vote['demCaucus']['caucus'] == 'no'):
        vote_class = 'neither'
    elif (vote['gopCaucus']['caucus'] == 'yes' and vote['demCaucus']['caucus'] == 'no'):
        vote_class = 'only GOP'
    elif (vote['gopCaucus']['caucus'] == 'no' and vote['demCaucus']['caucus'] == 'yes'):
        vote_class = 'only Dem'
    else:
        vote_class = 'error'
    
    return {
        'bill': vote['bill'],
        'bill_action': vote['bill_action'],
        'counts': vote['counts'],
        'gopCaucus': vote['gopCaucus'],
        'demCaucus': vote['demCaucus'],
        'vote_class': vote_class,
    }

inc_actions = ['2nd Reading Passed','2nd Reading Concurred', '2nd Reading Concurred as Amended', '2nd Reading Not Passed',
    '2nd Reading Not Concurred', '2nd Reading Passed as Amended', '2nd Reading Not Passed as Amended',
    '2nd Reading Pass Motion Failed', '2nd Reading Concur Motion Failed',
 ]

def get_bill(bill_id):
    return list(filter(lambda bill: bill['_id'] == bill_id, bills))

In [154]:
# get_bill('026accde-50c5-11e9-b623-0242ac160002')

In [150]:
def summarize_bill(bill):
    bill_votes = get_bill_votes(bill)
    categorized = [categorize_votes(vote) for vote in bill_votes if (vote['bill_action'] in inc_actions)]
#     cateogrized = [{'c': categorize_votes(vote)} for vote in bill_votes if (vote['bill_action'] in inc_actions)]
#     summarized = dict(Counter([action['vote_class'] for action in categorized]))
    
    return {
        'id': bill['_id'],
        'bill': bill['title'],
        'votes': tuple([action['vote_class'] for action in categorized]),
#         'categorized': categorized,
    }

summarize_bill(bills[10])

{'id': '026accde-50c5-11e9-b623-0242ac160002',
 'bill': 'Revise legislative objection polling for administrative rulemaking process',
 'votes': ('both support', 'both support')}

In [141]:
Counter([summarize_bill(bill)['votes'] for bill in passed])

Counter({('both support', 'both support'): 337,
         ('only GOP', 'only GOP'): 15,
         ('only Dem', 'both support'): 16,
         ('only Dem', 'only Dem', 'both support'): 2,
         ('both support', 'only GOP'): 16,
         ('only GOP', 'both support'): 10,
         ('both support', 'only Dem'): 12,
         ('both support', 'both support', 'only Dem'): 2,
         ('both support', 'only GOP', 'only GOP'): 1,
         ('both support', 'both support', 'both support'): 13,
         ('only Dem', 'only Dem'): 8,
         ('only Dem', 'both support', 'only Dem'): 3,
         ('only Dem', 'only GOP'): 1,
         (): 1,
         ('both support', 'both support', 'both support', 'both support'): 2,
         ('both support', 'only Dem', 'only Dem'): 1,
         ('only Dem', 'only Dem', 'only Dem'): 3,
         ('both support',): 1,
         ('both support', 'only Dem', 'both support'): 1,
         ('only GOP', 'only Dem'): 1})

In [137]:
# [vote for vote in votes if vote['bill'] == 'fc5631ce-50c6-11e9-b623-0242ac160002'][0]

In [160]:
[vote for vote in votes if get_bill(vote['bill'])['extras']['status'] == 'Became Law']

TypeError: list indices must be integers or slices, not str

In [ ]:
categorize_votes(vote)['vote_class']

In [162]:
votes_on_passed_bills = list(filter(lambda vote: get_bill(vote['bill'])['extras']['status'] == 'Became Law', bills))

KeyError: 'bill'